In [ ]:
for k in ["Chinese", "Malay", "Indian"]:
    for i in range(1,11,1):
        Regulons = "/rfs-storageservice/GIS/Projects/LOSBDA/Radhika/Final_Analysis/PB/30GRNBoost/" + k[0] + "R/T" + str(i) + "/reg_PB.csv"
        Output = "/rfs-storageservice/GIS/Projects/LOSBDA/Radhika/Final_Analysis/PB/30GRNBoost/" + k[0] + "R/T" + str(i) + "/AUCell_Output.loom"
        !pyscenic aucell \
       "/scratch/users/astar/gis/sonthaliar/Pyscenic/PB_All_SG_libs/PB_ExpMatrix_Sum_All.loom" \
        {Regulons} \
        --output {Output} \
        --num_workers 15

In [26]:
import loompy as lp 
import pandas as pd
import numpy as np
from scipy.stats import ranksums

##The working directory needs the PB (summed count activity matrix) in the format  DCP_ID__Annotations_lvl2__Sex__Ethnicity with its index

def Pipeline_WRS_ForMultipleEth_PB(auc_mtx, Trial_Number): 
    
    def p_adjust_bh(p):
        p = np.asfarray(p)
        by_descend = p.argsort()[::-1] #returns the index 
        by_orig = by_descend.argsort()
        steps = float(len(p)) / np.arange(len(p), 0, -1)
        q = np.minimum(1, np.minimum.accumulate(steps * p[by_descend]))
        return q[by_orig]

    def Wilcoxon_Rank_Sum (list_of_Regulons, Ethnicity): 
            AM = auc_mtx
            if Ethnicity != "All":
                S1 = AM[(AM["Sex"] == "Male") & (AM["Ethnicity"] == Ethnicity)]
                S2 = AM[(AM["Sex"] == "Female") & (AM["Ethnicity"] == Ethnicity)]
            else:
                S1 = AM[(AM["Sex"] == "Male")]
                S2 = AM[(AM["Sex"] == "Female")]
                
            finalContainerDictionary = {}    
            for i in list(set(S1["Annotations_lvl2"].unique()) & set(S2["Annotations_lvl2"].unique())): 
                replacements = {'+': 'positive', '-': 'negative'}
                replaced_chars = [replacements.get(char, char) for char in i] #removing + & - 
                new = ''.join(replaced_chars)
                locals()[ new + 'Dict_With_WRS'] = {} 
                for j in list_of_Regulons:
                    locals()[new + 'Dict_With_WRS'][j] = ranksums(S1[S1["Annotations_lvl2"] == i][j], S2[S2["Annotations_lvl2"] == i][j], alternative='two-sided')
                    Appending_letter = "TestPB"
                    tempDataframe = pd.DataFrame(locals()[new + 'Dict_With_WRS'])
                    tempDataframe = tempDataframe.T
                    tempDataframe.columns = ["Test_Statistic", "p value"]
                    tempDataframe["FDR Non-Global"] = p_adjust_bh(list(tempDataframe["p value"]))
                    finalContainerDictionary[new + Appending_letter + Ethnicity[0]] = tempDataframe
            
            return finalContainerDictionary

    List_With_MetaData_Info = [["Annotations_lvl2", 1], ["Sex", 2], ["DCP_ID", 0], ["Ethnicity", 3]]
    
    def Function_Metadata_Addition(subset):
        for i in range(len(List_With_MetaData_Info)):
            locals()[List_With_MetaData_Info[i][0]] = list()
            for k in subset.index:
                locals()[List_With_MetaData_Info[i][0]].append(k.split("__")[List_With_MetaData_Info[i][1]])
            subset[List_With_MetaData_Info[i][0]] = locals()[List_With_MetaData_Info[i][0]] 
    
    Function_Metadata_Addition(auc_mtx)    

    auc_mtx = auc_mtx[~auc_mtx["Annotations_lvl2"].isin(["pDC", "mDC", "Plasma B", "flagged_cluster"])] #only retaining main cell types
    Regulons = auc_mtx.filter(like='+', axis=1).columns


    #Running the pipeline for PB 
    PB_global = {}
    for Eth in ["Chinese", "Malay", "Indian", "All"]:
        if Eth != "All":
            Subset_Male = auc_mtx[(auc_mtx["Sex"] == "Male") & (auc_mtx["Ethnicity"] == Eth)]
            Subset_Female = auc_mtx[(auc_mtx["Sex"] == "Female") & (auc_mtx["Ethnicity"] == Eth)]
        else:
            Subset_Male = auc_mtx[auc_mtx["Sex"] == "Male"]
            Subset_Female = auc_mtx[auc_mtx["Sex"] == "Female"]
        
        DataframesPB = Wilcoxon_Rank_Sum(Regulons, Eth)
        
        list_Annotations_PB = list()
        list_of_p_values_PB = list()
        for i in list(set(Subset_Male["Annotations_lvl2"].unique()) & set(Subset_Female["Annotations_lvl2"].unique())):
                replacements = {'+': 'positive', '-': 'negative'}
                replaced_chars = [replacements.get(char, char) for char in i] #removing + & - 
                new = ''.join(replaced_chars)
                list_Annotations_PB.append(new + "TestPB" + Eth[0])
                list_of_p_values_PB.append((DataframesPB[new + "TestPB" + Eth[0]]["p value"]))
        flat_list = [item for sublist in list_of_p_values_PB for item in sublist] ##Using the p values from flat list > we know index 24 = ILC
        Adjusted_p_PB = p_adjust_bh(flat_list)
        for CellType in list_Annotations_PB:
            DataframesPB[CellType]["FDR Global"] = np.array_split(Adjusted_p_PB, len(set(Subset_Male["Annotations_lvl2"].unique()) & set(Subset_Female["Annotations_lvl2"].unique())))[list_Annotations_PB.index(CellType)] ##split into 25 parts, index = 24 ILC
        PB_global.update(DataframesPB)
    
    return PB_global


In [40]:


import math
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import matplotlib as mpl
from matplotlib import ticker as mtick
from matplotlib import ticker
# plt.rcParams['font.serif'] = ['Times New Roman']
def AUC_Histogram_PB(auc_mtx, Regulons, Cell_Type, Ethnicity, nbins, Trial_Number, Regulons_from, Regulon_Label_Size, pvalSize):
    sns.set_theme( rc={
    'xtick.bottom': True,
    'ytick.left': True,})
    
#     csfont = {'fontname':'Arial'}
    def fmt_SF_Digits(x, pos):
        return f'{x:.4f}'
    Dict = PB
    palette_updated ={"Male": "blue", "Female": "#C51B8A"}
    replacements = {'+': 'positive', '-': 'negative'}
    replaced_chars = [replacements.get(char, char) for char in Cell_Type] #removing + & -
    new = ''.join(replaced_chars)
    new_updated = new + "TestPB"
    Regulons = Regulons
    temp =  math.ceil(len(Regulons)/2) #gives you the number of rows!
    
    fig, axes = plt.subplots(nrows=(2*int(temp)), ncols=2, figsize=(17, (temp*12.5)), gridspec_kw={"height_ratios": (int(temp))*[1,3]}) #2x3 , you want 1 row & 3 columns
    #gs = fig.add_gridspec(nrows=2*int(temp), ncols= 4,  gridspec_kw={"height_ratios": 2*int(temp)*[1,3]})
    fig.suptitle(Cell_Type + "_" + "PB input" + "_" + Trial_Number + "_" + Regulons_from + "_" + Ethnicity , fontsize = 30, y = 0.95)
#    fig.suptitle("Sample " + "Regulon Label = " + str(Regulon_Label_Size) + ",p value label signs = " + str(pvalSize) , fontsize = 30, y = 0.95)
    df = auc_mtx[(auc_mtx["Annotations_lvl2"] == Cell_Type) & (auc_mtx["Ethnicity"] == Ethnicity)]
    
    count_1 = -1
    count_2 = -1
    for i in range(0,(2*int(temp)),1):
        for j in range(0,4,1):
            if (i % 2) == 0 :
                count_1 += 1
                if count_1 < (len(Regulons)):
                    min_val = df[Regulons[count_1]].min()
                    max_val = df[Regulons[count_1]].max()
                    val_width = max_val - min_val
                    n_bins = 10
                    bin_width = val_width/n_bins
        
                    plot_box = sns.boxplot(ax = axes[i,j], x = df[Regulons[count_1]], y = df["Sex"], hue = df["Sex"], palette = palette_updated, boxprops=dict(alpha=.3), order=["Male", "Female"], flierprops={'marker': 'o'}, hue_order=["Male", "Female"])
                    plot_box.get_legend().remove()
                    axes[i,j].tick_params(axis = "x", labelrotation = 45, bottom = "on", labelsize = "large")
                    axes[i,j].tick_params(axis = "y",  labelsize = "large")
                    plot_box.set_ylabel("Sex", fontsize=20)
                    #plot_box.set(xlabel=None)
                    plot_box.set_xlabel(Regulons[count_1],fontsize=Regulon_Label_Size)
                    plt.setp(axes[i,j], xticks=np.arange(min_val-bin_width/2, max_val+bin_width/2, bin_width), visible = True)
                    axes[i,j].xaxis.set_major_formatter(ticker.FuncFormatter(fmt_SF_Digits))

            else:
                count_2 += 1
                if count_2 < (len(Regulons)):
                    min_val = df[Regulons[count_2]].min()
                    max_val = df[Regulons[count_2]].max()
                    n_bins = 10
                    val_width = max_val - min_val
                    bin_width = val_width/n_bins
                    plot_a = sns.histplot(ax = axes[i,j],data =  df, x = Regulons[count_2], hue='Sex', stat = 'percent', common_norm = False, palette= palette_updated, bins = nbins, legend = False, hue_order = ["Male", "Female"], multiple = "dodge", shrink=0.6)
#                     plot_a.suptitle("Trial",  fontsize = 30, y = 0.92)
                    plot_a.set_xlabel(Regulons[count_2] ,fontsize=Regulon_Label_Size)
                    plot_a.set_ylabel("Percent" ,fontsize=20)
                    pval = "p-value = " + str("{:.2e}".format(Dict[new_updated + Ethnicity[0]].T[Regulons[count_2]]["p value"]))
                    FDRGlobal = "FDR = " + str("{:.2e}".format(Dict[new_updated + Ethnicity[0]].T[Regulons[count_2]]["FDR Global"]))
                    FDR_NonGlobal = "Regulon-specific FDR = " + str("{:.2e}".format(Dict[new_updated + Ethnicity[0]].T[Regulons[count_2]]["FDR Non-Global"]))
                    string_with_info = '\n'.join([str(pval), str(FDRGlobal), str(FDR_NonGlobal)])
#                     plt.text(1,1, string_with_info, ha = 'right', va = 'top', transform=axes[i,j].transAxes, fontsize = 15 )
                    plot_a.tick_params(axis = "x", labelsize = "large", labelrotation = 45) #direction = "out", length = 10, width = 10, colors = "r")
                    plt.subplots_adjust(wspace=0.2,hspace= 0.6)
#                     axes[i,j].title(label = "Trial")
                    plot_a.set_title(string_with_info, loc = "right", fontsize = pvalSize)
                    plot_a.tick_params(axis = "y",  labelsize = "large")
                    plt.setp(axes[i,j], xticks=np.arange(min_val-bin_width/2, max_val+bin_width/2, bin_width), visible = True)
                    axes[i,j].xaxis.set_major_formatter(ticker.FuncFormatter(fmt_SF_Digits))



            fig.savefig("/rfs-storageservice/GIS/Projects/LOSBDA/Radhika/Final_Analysis/PB/30GRNBoost/" + Regulons_from + "/Plots/"  + Cell_Type + "_" + Trial_Number + "_" + Ethnicity + ".png")


In [ ]:
for Regulons in ["CR", "MR", "IR"]:
    for trial in range(1,11,1):
        lf = lp.connect( "/rfs-storageservice/GIS/Projects/LOSBDA/Radhika/Final_Analysis/PB/30GRNBoost/" + Regulons + "/T" + str(trial) + "/AUCell_Output.loom", mode='r', validate=False )
        auc_mtx = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
        lf.close()
        Rownames = pd.read_csv("/scratch/users/astar/gis/sonthaliar/Pyscenic/Regulons_Diff_Trial_Chinese/Rownames.csv")
        auc_mtx.index = Rownames["Pseudobulk"]
        PB = Pipeline_WRS_ForMultipleEth_PB(auc_mtx, str(trial))
        for Eth in ["Chinese", "Malay", "Indian"]:
            AUC_Histogram_PB(auc_mtx, ["ZBTB7A(+)", "SP1(+)"], "Treg", Eth, 10, str(trial), Regulons, 25, 20)


<function __main__.AUC_Histogram_Trial_PB(auc_mtx, Regulons, Cell_Type, Ethnicity, nbins, Trial_Number, Regulons_from, Regulon_Label_Size, pvalSize)>